# Writing the pair momentum distribution in terms of single-nucleon momentum distributions

__Author:__ A. J. Tropiano [atropiano@anl.gov]<br/>
__Date:__ July 4, 2023

We compute pair momentum distributions in terms of single-nucleon momentum distributions.

_Last update:_ July 4, 2023

In [ ]:
# Python imports
from matplotlib.offsetbox import AnchoredText
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy.polynomial.legendre import leggauss
from scipy.interpolate import InterpolatedUnivariateSpline

In [ ]:
# Imports from scripts
from scripts.integration import gaussian_quadrature_mesh

from test_momentum_distribution_script import load_momentum_distribution
from test_pmd_script import compute_normalization as pmd_norm

## Set-up

In [ ]:
# Run this cell to turn on customized matplotlib graphics
set_rc_parameters()

## Derivation

_add derivation here_

## Testing

In [ ]:
def n_pair(
        nucleus_name, pair, kvnn, lamb, Q_max=5.0, ntot_Q=50,
        print_normalization=False
):
    """Pair momentum distribution in terms of single-nucleon momentum
    distributions.
    """
    
    if pair == 'pp':
        nucleon = 'proton'
        nucleonp = 'proton'
    elif pair == 'nn':
        nucleon = 'neutron'
        nucleonp = 'neutron'
    elif pair == 'pn':
        nucleon = 'proton'
        nucleonp = 'neutron'
    elif pair == 'np':
        nucleon = 'neutron'
        nucleonp = 'proton'

    # C.o.M. momenta
    Q_array, Q_weights = gaussian_quadrature_mesh(Q_max, ntot_Q)
    
    # Load single-nucleon momentum distributions
    (q_array, q_weights, n_tau_array, _, _, _, _, _, _) = (
        load_momentum_distribution(nucleus_name, nucleon, kvnn, lamb)
    )
    (q_array, q_weights, n_taup_array, _, _, _, _, _, _) = (
        load_momentum_distribution(nucleus_name, nucleonp, kvnn, lamb)
    )
    
    # Interpolate single-nucleon momentum distributions
    n_tau_func = InterpolatedUnivariateSpline(q_array, n_tau_array)
    n_taup_func = InterpolatedUnivariateSpline(q_array, n_taup_array)
    
    # Integration for angle between q_vector and Q_vector
    x_array, x_weights = leggauss(15)
    
    # Get 3-D meshgrid and integrate evaluate n(q_vector, Q_vector)
    q_grid, Q_grid, x_grid = np.meshgrid(q_array, Q_array, x_array,
                                         indexing='ij')
    _, _, dx_grid = np.meshgrid(q_array, Q_array, x_weights, indexing='ij')
    
    # |Q/2+q|
    q1_grid = np.sqrt(Q_grid ** 2 / 4 + q_grid ** 2 + q_grid * Q_grid * x_grid)
    q2_grid = np.sqrt(Q_grid ** 2 / 4 + q_grid ** 2 - q_grid * Q_grid * x_grid)
    
    # n^\tau(|Q/2+q|)
    n_tau_grid_x = n_tau_func(q1_grid)
    # n^\tau'(|Q/2-q|)
    n_taup_grid_x = n_taup_func(q2_grid)
    
    # Average over x
    n_tau_grid = np.sum(n_tau_grid_x * dx_grid, axis=-1) / 2
    n_taup_grid = np.sum(n_taup_grid_x * dx_grid, axis=-1) / 2
    
    # Append q=0 term?
    if tau == taup:
        # TO-DO!
        pass
    
    # 1/2 n^\tau'(q,Q) n^\tau(q,Q)
    n_grid = 1/2 * n_tau_grid * n_taup_grid
    
    if print_normalization:
        normalization = compute_normalization(q_array, q_weights, Q_array,
                                              Q_weights, n_grid)
        print(f"Normalization = {normalization:.5f}.")
    
    return q_array, q_weights, Q_array, Q_weights, n_grid

In [ ]:
# He4 pn
n_pair('He4', 'pn', 6, 1.5, print_normalization=True)